# Setup Environment

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CS505")

# Read Dataset

In [0]:
import pandas as pd

dataset = pd.read_pickle('dataset_new.pkl')

# Data Preprocessing

In [0]:
import numpy as np

dataset['id'] = dataset.iloc[:,0].map(lambda x: x[1])
dataset['dates'] = dataset.iloc[:,1].map(lambda x: x[1])
dataset['transaction_amount'] = dataset.iloc[:,2].map(lambda x: x[1])
dataset['days_before_request'] = dataset.iloc[:,3].map(lambda x: x[1])
dataset['loan_amount'] = dataset.iloc[:,4].map(lambda x: x[1])
dataset['loan_date'] = dataset.iloc[:,5].map(lambda x: x[1])
dataset['isDefault'] = dataset.iloc[:,6].map(lambda x: np.NaN if len(x)==0 else x[1])
dataset.drop(columns=[0, 1, 2, 3, 4, 5, 6], inplace=True)

# Feature Extraction

In [0]:
features = []
last_transaction = 30
for index, row in dataset.iterrows():
  transaction_amount = row['transaction_amount']
  cumsum = np.cumsum(transaction_amount)
  cumsum_balance = cumsum[-1] - cumsum[0]
  loan_amount = row['loan_amount']

  credit_max = np.max(transaction_amount)
  credit_count = np.sum(transaction_amount>0)
  credit_transaction_amount = transaction_amount[transaction_amount>0]
  if np.all(credit_transaction_amount!=credit_transaction_amount):
    credit_mean = 0
    credit_median = 0
    credit_var = 0 
  else:
    credit_mean = np.mean(credit_transaction_amount)
    credit_median = np.median(credit_transaction_amount)
    credit_var = np.var(credit_transaction_amount)

  debit_min = np.min(transaction_amount)
  debit_count = np.sum(transaction_amount<0)
  debit_transaction_amount = transaction_amount[transaction_amount<0]
  if np.all(debit_transaction_amount!=debit_transaction_amount):
    debit_mean = 0
    debit_median = 0
    debit_var = 0
  else:
    debit_mean = np.mean(debit_transaction_amount)
    debit_median = np.median(debit_transaction_amount)
    debit_var = np.var(debit_transaction_amount)

  dates = pd.to_datetime(row['dates'])
  quarter_df = pd.DataFrame({'transaction_amount': transaction_amount.ravel()}, index=dates).resample('Q')
  last_quarter_count = quarter_df.count().iloc[-1,0]
  last_quarter_total = quarter_df.sum().iloc[-1,0]
  last_quarter_max = quarter_df.max().iloc[-1,0]
  last_quarter_min = quarter_df.min().iloc[-1,0]
  last_quarter_mean = quarter_df.mean().iloc[-1,0]
  last_quarter_median = quarter_df.median().iloc[-1,0]
  last_quarter_var = quarter_df.var().iloc[-1,0]

  if len(transaction_amount)>last_transaction:
    last_transaction_amount = transaction_amount[-last_transaction:]
    total = np.sum(last_transaction_amount)
    last_cumsum = np.cumsum(last_transaction_amount)
    last_cumsum_balance = last_cumsum[-1] - last_cumsum[0]
    last_credit_max = np.max(last_transaction_amount)
    last_credit_count = np.sum(last_transaction_amount>0)
    last_credit_transaction_amount = last_transaction_amount[last_transaction_amount>0]
    if np.all(last_credit_transaction_amount!=last_credit_transaction_amount):
      last_credit_mean = 0
      last_credit_median = 0
      last_credit_var = 0
    else:
      last_credit_mean = np.mean(last_credit_transaction_amount)
      last_credit_median = np.median(last_credit_transaction_amount)
      last_credit_var = np.var(last_credit_transaction_amount)

    last_debit_min = np.min(last_transaction_amount)
    last_debit_count = np.sum(last_transaction_amount<0)
    last_debit_transaction_amount = last_transaction_amount[last_transaction_amount<0]
    if np.all(last_debit_transaction_amount!=last_debit_transaction_amount):
      last_debit_mean = 0
      last_debit_median = 0
      last_debit_var = 0
    else:
      last_debit_mean = np.mean(last_debit_transaction_amount)
      last_debit_median = np.median(last_debit_transaction_amount)
      last_debit_var = np.var(last_debit_transaction_amount)
  else:
    last_total = total
    last_cumsum_balance = cumsum_balance
    last_credit_max = credit_max
    last_credit_count = credit_count
    last_credit_mean = credit_mean
    last_credit_median = credit_median
    last_credit_var = credit_var
    last_debit_min = debit_min
    last_debit_count = debit_count
    last_debit_mean = debit_mean
    last_debit_median = debit_median
    last_debit_var = debit_var

  features.append([total, cumsum_balance, loan_amount,
                   credit_max, credit_count, credit_mean, credit_median, credit_var,
                   debit_min, debit_count, debit_mean, debit_median, debit_var,
                   last_quarter_count, last_quarter_total,
                   last_quarter_max, last_quarter_min, last_quarter_mean, last_quarter_median, last_quarter_var,
                   last_cumsum_balance,
                   last_credit_max, last_credit_count, last_credit_mean, last_credit_median, last_credit_var,
                   last_debit_min, last_debit_count, last_debit_mean, last_debit_median, last_debit_var])

features=np.array(features)
features[np.isnan(features)] = 0

In [0]:
df = pd.DataFrame(features)
df.to_csv('extracted_features.csv')

In [0]:
import numpy as np

features = pd.read_csv('extracted_features.csv')

# Data Preparation

In [0]:
X_data = features[:10500]
y_data = dataset['isDefault'].iloc[:10500]
X_test = features[10000:]

I have tested many different scalers (MinMaxScaler [0, 1] or [-1, 1], RobustScaler), it seems that only Standardize provides best performance. Moreover, using scaled data can reduce the time for training and more importantly it's better to scale data before feeding to neural network.

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)
X_test = scaler.transform(X_test)

## Feature Transformation

The following two methods were tested, however there is no significant score increase.

In [0]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=10).fit(X_train)
dist_train = np.column_stack([np.sum((X_train - center)**2, axis=1)**0.5 for center in kmeans.cluster_centers_])
dist_val = np.column_stack([np.sum((X_val - center)**2, axis=1)**0.5 for center in kmeans.cluster_centers_])
dist_test = np.column_stack([np.sum((X_test - center)**2, axis=1)**0.5 for center in kmeans.cluster_centers_])

In [0]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=10).fit(X_train)
proba_train = gmm.predict_proba(X_train)
proba_val = gmm.predict_proba(X_val)
proba_test = gmm.predict_proba(X_test)

## Imbalanced Data

As the dataset is imbalanced, there are several methods to deal with that.

### Up-sample Minority Class

In [0]:
from sklearn.utils import resample

# Separate majority and minority classes
majority = X_train[y_train==0]
minority = X_train[y_train==1]

# Upsample minority class
minority_upsampled = resample(minority, n_samples=len(majority))

# Combine majority class with upsampled minority class
X_train_upsampled = np.concatenate((majority, minority_upsampled))
y_train_upsampled = np.concatenate((np.zeros(len(majority)), np.ones(len(majority))))

### Down-sample Majority Class

In [0]:
from sklearn.utils import resample

# Separate majority and minority classes
majority = X_data[y_label==0]
minority = X_data[y_label==1]

# Downsample majority class
majority_downsampled = resample(majority, replace=False, n_samples=len(minority))

# Combine majority class with upsampled minority class
X_data_downsampled = np.concatenate((majority_downsampled, minority))
y_label = np.concatenate((np.zeros(len(minority)), np.ones(len(minority))))

# Statistical Learning

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [0]:
def print_results(results):
    print('Best Params: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

I have tested all kinds of different algorithms in sklearn, but only keep few of them as example. It shows as following and not all the details are included as the code is kind of massy during hyper-parameter tuning.

## Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs', class_weight='balanced', n_jobs=-1, verbose=1)
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

cv = GridSearchCV(lr, parameters, cv=10, n_jobs=-1, verbose=1)
cv.fit(X, y_train)

In [0]:
print_results(cv)

Best Params: {'C': 0.001}

0.625 (+/-0.023) for {'C': 0.001}
0.625 (+/-0.024) for {'C': 0.01}
0.623 (+/-0.023) for {'C': 0.1}
0.622 (+/-0.024) for {'C': 1}
0.622 (+/-0.024) for {'C': 10}
0.622 (+/-0.024) for {'C': 100}
0.622 (+/-0.024) for {'C': 1000}


In [0]:
preds_proba = cv.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, preds_proba)

0.6159314113733566


## SVM

In [0]:
from sklearn.svm import SVC

svc = SVC(gamma='scale', class_weight='balanced', probability=True, verbose=1)
parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

cv = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 17.9min finished


[LibSVM]

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight='balanced',
                           coef0=0.0, decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=1),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [0]:
print_results(cv)

Best Params: {'C': 10, 'kernel': 'rbf'}

0.564 (+/-0.035) for {'C': 0.1, 'kernel': 'linear'}
0.574 (+/-0.041) for {'C': 0.1, 'kernel': 'rbf'}
0.564 (+/-0.036) for {'C': 1, 'kernel': 'linear'}
0.594 (+/-0.038) for {'C': 1, 'kernel': 'rbf'}
0.564 (+/-0.035) for {'C': 10, 'kernel': 'linear'}
0.595 (+/-0.039) for {'C': 10, 'kernel': 'rbf'}


In [0]:
preds_proba = cv.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, preds_proba)

0.6478070134565922

## Multi-Layer Perceptron

In [0]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(verbose=1)
parameters = {
    'hidden_layer_sizes' : [(10,), (50,), (100,)],
    'activation' : ['relu', 'tanh', 'logistic'],
    'learning_rate' : ['constant', 'invscaling', 'adaptive']
}

cv = GridSearchCV(mlp, parameters, cv=10, n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

In [0]:
print_results(cv)

Best Params: {'activation': 'logistic', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}

0.54 (+/-0.082) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.541 (+/-0.035) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.536 (+/-0.061) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.537 (+/-0.038) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.539 (+/-0.046) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.539 (+/-0.048) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.538 (+/-0.023) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.532 (+/-0.058) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.529 (+/-0.036) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'lea

In [0]:
preds_proba = cv.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, preds_proba)

## Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(verbose=1, n_jobs=-1, class_weight='balanced')
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth' : [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=10, n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.0min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


180.68671035766602


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.7s finished


In [0]:
print_results(cv)

Best Params: {'max_depth': 32, 'n_estimators': 50}

0.558 (+/-0.059) for {'max_depth': 2, 'n_estimators': 5}
0.558 (+/-0.04) for {'max_depth': 2, 'n_estimators': 50}
0.56 (+/-0.037) for {'max_depth': 2, 'n_estimators': 250}
0.583 (+/-0.041) for {'max_depth': 4, 'n_estimators': 5}
0.585 (+/-0.055) for {'max_depth': 4, 'n_estimators': 50}
0.58 (+/-0.039) for {'max_depth': 4, 'n_estimators': 250}
0.603 (+/-0.026) for {'max_depth': 8, 'n_estimators': 5}
0.618 (+/-0.047) for {'max_depth': 8, 'n_estimators': 50}
0.62 (+/-0.046) for {'max_depth': 8, 'n_estimators': 250}
0.66 (+/-0.032) for {'max_depth': 16, 'n_estimators': 5}
0.693 (+/-0.023) for {'max_depth': 16, 'n_estimators': 50}
0.691 (+/-0.015) for {'max_depth': 16, 'n_estimators': 250}
0.688 (+/-0.026) for {'max_depth': 32, 'n_estimators': 5}
0.715 (+/-0.016) for {'max_depth': 32, 'n_estimators': 50}
0.714 (+/-0.018) for {'max_depth': 32, 'n_estimators': 250}
0.681 (+/-0.018) for {'max_depth': None, 'n_estimators': 5}
0.712 (+/-0.017) 

In [0]:
preds_proba = cv.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, preds_proba)

0.5478342714429493


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.1s finished


## Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(verbose=1)
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth' : [1, 3, 5, 7, 9],
    'learning_rate' : [0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(clf, parameters, cv=10, n_jobs=-1, verbose=1)
cv.fit(X_train, y_train)

In [0]:
print_results(cv)

Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}

0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5}
0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}
0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500}
0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5}
0.73 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
0.731 (+/-0.002) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250}
0.733 (+/-0.002) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
0.731 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}
0.731 (+/-0.001) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
0.73 (+/-0.006) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 250}
0.728 (+/-0.005) for {'learning_rate': 0.01, 'ma

In [0]:
preds_proba = cv.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, preds_proba)

0.5083039275212871


# Deep Learning

## Feature Selection

As tree algorithms are robust to imbalanced dataset, we could use it to safely drop less important features to increase the score for neural network. Below GradientBoostingClassifier provides us about auc=0.68 which is an acceptable score, thus we use this classifier to select our features.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

clf = GradientBoostingClassifier(learning_rate=0.01, max_depth=7, n_estimators=2000, validation_fraction=0.2, n_iter_no_change=200, max_leaf_nodes=150, verbose=1)
clf = clf.fit(X_data, y_data)

      Iter       Train Loss   Remaining Time 
         1           1.1684            2.49m
         2           1.1649            2.51m
         3           1.1615            2.52m
         4           1.1582            2.51m
         5           1.1549            2.55m
         6           1.1518            2.59m
         7           1.1486            2.61m
         8           1.1457            2.60m
         9           1.1427            2.58m
        10           1.1399            2.57m
        20           1.1133            2.53m
        30           1.0898            2.53m
        40           1.0684            2.52m
        50           1.0492            2.52m
        60           1.0313            2.51m
        70           1.0146            2.50m
        80           0.9994            2.49m
        90           0.9855            2.48m
       100           0.9711            2.48m
       200           0.8661            2.30m
       300           0.7973            2.11m


In [13]:
clf.feature_importances_

array([0.01341585, 0.01457309, 0.01508406, 0.02192532, 0.03729395,
       0.1553824 , 0.02540418, 0.02544947, 0.02020085, 0.03539481,
       0.02741562, 0.01621815, 0.08658484, 0.01974588, 0.0516457 ,
       0.02586026, 0.02964507, 0.02375072, 0.02153698, 0.02148523,
       0.01851402, 0.03742926, 0.02379334, 0.02811353, 0.02243695,
       0.01059875, 0.01746469, 0.02291281, 0.02252937, 0.0147584 ,
       0.00595606, 0.03856867, 0.0292592 , 0.01965254])

In [0]:
model = SelectFromModel(clf, prefit=True)
X_data = model.transform(X_data)
X_test = model.transform(X_test)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2)

## Neural Network
As I have tried, the best algorithm in sklearn is gradient boosting which can provide auc=0.68 in cross validation. Thus, we try to use deep learning method to increase the score.

In [0]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weight_dict = dict(enumerate(class_weights))

In [15]:
import tensorflow as tf
from keras import backend as K
def auc(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import *
from keras.constraints import maxnorm
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# Function to create model
def create_model(optimizer='adam', init='glorot_uniform'):
  # create model
  model = Sequential()
  model.add(Dropout(0.2, input_shape=(X_train.shape[1],)))
  model.add(Dense(64, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(128, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(64, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

  # Compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[auc])
  return model

If we use GridSearchCV and KerasClassifier Wrapper for hyper-parameters tuning with sklearn, it won't use GPU to accelarate the process. To speed up the process, we just loop through parameters' arrays with 10-fold cross validation (really time consuming). We couldn't search for the best hyper-parameters due limited time.

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam', 'sgd']
inits = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
for optimizer in optimizers:
	for init in inits:
		for epoch in epochs:
			for batch in batches:
				model = create_model(optimizer=optimizer, init=init)
				# split into input (X) and output (Y) variables
				X = X_train
				Y = y_train
				# define 10-fold cross validation test harness
				kfold = StratifiedKFold(n_splits=10, shuffle=True)
				cvscores = []
				for train, test in kfold.split(X, Y):
					# create model
					model = create_model()
					# Fit the model
					model.fit(X[train], Y[train], epochs=epoch, batch_size=batch, class_weight=class_weight_dict, verbose=0)
					# evaluate the model
					preds_proba = model.predict_proba(X[test])
					scores = roc_auc_score(Y[test], preds_proba)
					cvscores.append(scores)
				print("%s / %s / %d / %d -- %.3f (+/- %.3f)" % (optimizer, init, epoch, batch, np.mean(cvscores), np.std(cvscores))) 

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# split into input (X) and output (Y) variables
X = X_train
Y = y_train
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
for train, test in kfold.split(X, Y):
  # create model
  model = create_model()
  # Fit the model
  model.fit(X[train], Y[train], epochs=200, class_weight=class_weight_dict, verbose=0)
  # evaluate the model
  preds_proba = model.predict_proba(X[test])
  scores = roc_auc_score(Y[test], preds_proba)
  print("%s: %.3f" % ('auc', scores))
  cvscores.append(scores)
print("%.3f (+/- %.3f)" % (np.mean(cvscores), np.std(cvscores))) 

auc: 63.58
auc: 62.03
auc: 67.23
auc: 64.96
auc: 67.02
auc: 65.64
auc: 63.80
auc: 64.93
auc: 64.07
auc: 66.62
64.99 (+/- 1.59)


In [0]:
from keras.models import Sequential
from keras.layers import *
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

def create_model(init='glorot_uniform'):
  # create model
  model = Sequential()
  model.add(Dropout(0.2, input_shape=(X_train.shape[1],)))
  model.add(Dense(32, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(64, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(32, kernel_initializer=init, activation='relu', kernel_constraint=maxnorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

  # Compile model
  #sgd = SGD(lr=0.1, momentum=0.9)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
  return model

In [39]:
model = create_model()
model.fit(X_train, y_train, epochs=20, class_weight=class_weight_dict, verbose=1)

Epoch 1/20
8400/8400 [==============================] - 3s 303us/step - loss: 0.6873 - auc: 0.5558
Epoch 2/20
8400/8400 [==============================] - 1s 159us/step - loss: 0.6752 - auc: 0.5871
Epoch 3/20
8400/8400 [==============================] - 1s 157us/step - loss: 0.6675 - auc: 0.5999
Epoch 4/20
8400/8400 [==============================] - 1s 156us/step - loss: 0.6684 - auc: 0.6083
Epoch 5/20
8400/8400 [==============================] - 1s 163us/step - loss: 0.6666 - auc: 0.6121
Epoch 6/20
8400/8400 [==============================] - 1s 159us/step - loss: 0.6676 - auc: 0.6157
Epoch 7/20
8400/8400 [==============================] - 1s 155us/step - loss: 0.6617 - auc: 0.6195
Epoch 8/20
8400/8400 [==============================] - 1s 157us/step - loss: 0.6599 - auc: 0.6218
Epoch 9/20
8400/8400 [==============================] - 1s 160us/step - loss: 0.6598 - auc: 0.6249
Epoch 10/20
8400/8400 [==============================] - 1s 158us/step - loss: 0.6628 - auc: 0.6268
Epoch 11/

In [22]:
from sklearn.metrics import roc_auc_score
preds_proba = model.predict_proba(X_val)
roc_auc_score(y_val, preds_proba)

0.695259330786252

In [40]:
from sklearn.metrics import roc_auc_score
preds_proba = model.predict_proba(X_val)
roc_auc_score(y_val, preds_proba)

0.6915981370652332

In [31]:
model = create_model()
model.fit(X_data, y_data, epochs=10, class_weight=class_weight_dict, verbose=1)

Epoch 1/10
10500/10500 [==============================] - 3s 241us/step - loss: 0.6728 - auc: 0.5885
Epoch 2/10
10500/10500 [==============================] - 2s 160us/step - loss: 0.6657 - auc: 0.6261
Epoch 3/10
10500/10500 [==============================] - 2s 157us/step - loss: 0.6638 - auc: 0.6309
Epoch 4/10
10500/10500 [==============================] - 2s 158us/step - loss: 0.6616 - auc: 0.6345
Epoch 5/10
10500/10500 [==============================] - 2s 160us/step - loss: 0.6644 - auc: 0.6363
Epoch 6/10
10500/10500 [==============================] - 2s 157us/step - loss: 0.6558 - auc: 0.6399
Epoch 7/10
10500/10500 [==============================] - 2s 159us/step - loss: 0.6568 - auc: 0.6416
Epoch 8/10
10500/10500 [==============================] - 2s 162us/step - loss: 0.6566 - auc: 0.6434
Epoch 9/10
10500/10500 [==============================] - 2s 160us/step - loss: 0.6567 - auc: 0.6449
Epoch 10/10
10500/10500 [==============================] - 2s 163us/step - loss: 0.6562 - a

In [0]:
y_preds = model.predict_proba(X_test)

In [0]:
df = pd.DataFrame([i for i in range(10000, 15000)])
df[1] = y_preds
df.to_csv('output.csv', header=False, index=False)